In [3]:
import numpy as np
import graphviz

In [22]:
!dot -Tpng -O diagraph.txt

![diagraph.png](./diagraph.txt.png)

In [ ]:
# Define some parameters of the simulation
sim_par = {'N_sol': 20}

As a convention, J_ij is the link that goes from node j to node i

In [ ]:
# I try to create a class for the NN
class Network:
    """
    Class for a generic neural network, defined by 
    - 2 input neurons, 1 output neuron
    - S: # of intra neurons.
    - Theta: threshold value.
    - J: connectivity matrix.
    """
    
    def __init__(self, S):
        self.S = S
        self.Theta = np.zeros(2+self.S+1)   # I instantiate a threshold value for each neuron, even the input, just for consistency of the indexes
        # Define the connectivity matrix J0_ij
        J0 = np.ones((2+self.S+1,2+self.S+1)) # Initialized all to one, so all connnected
        for j in range(len(J0)):            # First, impose on J0 that the elements on the diagonal must be zero
            J0[j,j] = 0.
        J0[0,1] = 0.            # Impose on the connectiviy matrix the fact that nuerons on the same layer cannot be connected 
        J0[1,0] = 0.
        for i in range(2, 2 + self.S):
            for j in range(2, 2 + self.S):
                if i != j:
                    J0[i, j] = 0.       
        self.J = J0
        self.activation = np.heaviside()    # FIX!!!!
        self.neurons = np.zeros(2+self.S+1) # container for storing the values of the neurons
        
    def generate(self,par:dict):
        """Generate a random network.

        Args:
            par (dict, optional): parameters of the generation.
        """
        np.random.seed(par['seed'])     # set a seed
        self.S = par['Sk_range'][np.random.randint(len(par['Sk_range']))]   # Generate S
        self.Theta = np.random.uniform(par['Theta_range'][0],par['Theta_range'][1],2+self.S+1) # Generate the set of thresholds, Theta
        self.J = self.J * np.random.uniform(par['J_range'][0],par['J_range'][1],(2+self.S+1)**2).reshape((2+self.S+1,2+self.S+1))   # Generate the connectivity matrix, J
        
        return self
        
    def ff(self,input:list,verb:int=0): # feed-forward
        """Compute the output of the network by feed-forward, given an input.

        Args:
            input (list): input to the network. Supported inputs are [0,0];[0,1];[1,0];[1,1].
            verb (int,optional): verbosity. If > 0, returns the value of each neuron. Default to 0.

        Returns:
            output (int): output of the network.
        """
        self.neurons[0:2] = input   # set the value of the first two neurons to the input value
        
        for neuron in range(1,2+self.S):    # compute the value of the intra neurons by feed forward
            self.neurons[neuron] = self.J[neuron,0] * self.neurons[0] + self.J[neuron,1] * self.neurons[1]   # I refer to the lower triangle of the matrix J
            self.neurons[neuron] = self.activation(self.neurons[neuron]-self.Theta[neuron])     # Activation
        
        for neuron in range(0,2+self.S):  # compute the value of the ouput
            self.neurons[-1] = self.J[-1,neuron] * self.neurons[neuron]     # Add each neuron's weighted contribution to the output
        self.neurons[-1] = self.activation(self.neurons[-1]-self.Theta[-1])     # activation
        output = self.neurons[-1]
        if verb > 0:
            return self.neurons
        return output
        
    def fitness(self,input:list,target:int):
        output = self.ff(input)
        squared_dist = (target - output)**2
        squared_cost = (np.sum([0 if np.isclose(j, 0., rtol=1e-3) else 1 for j in self.J.flatten()])/2)**2  # check if the tolerance is good
        return squared_dist * squared_cost
    

For this model I don't need to define the space grid, I just need to generate the solutions and then execute the algorithm. I just need to make sure that they stay in the range.

I try first one loop:

In [ ]:
# Generate a set of parameters
par = {'Sk_range': [1,2,3,4,5,6,7,8,9],  # define the possible value for Sk
       'Theta_range': [-1,+1],
       'J_range': [-1,+1],
       'seed': 12345}

In [ ]:
# Generate the solutions
solutions = []
for _ in range(sim_par['N_sol']):
    # Generate parameters
    par['S'] = Sk_range[np.random.randint(len(Sk_range))]
    par['Theta'] = np.random.uniform(-1,+1,1)
    Jk = J0 * np.random.uniform(-1,+1,25).reshape((5,5))
    # Create the network associated with the generated parameters and add it to the solutions list
    solutions.append(Network(par,Jk))
    